In [ ]:
!git clone "https://github.com/smartyfh/MultiWOZ2.4.git"

In [ ]:
%cd "MultiWOZ2.4"
!python create_data.py
%cd ".."

In [ ]:
from prompt import eval_prompt_v3, eval_prompt_v3_reduced_objective
from evaluator import evaluator
from openai import OpenAI, AzureOpenAI
from getpass import getpass
import json
import random

with open("MultiWOZ2.4/data/mwz2.4/test_dials.json", "r") as f:
    data = json.load(f)

with open("MultiWOZ2.4/data/mwz24/data.json", "r") as f:
    old_data = json.load(f)

In [2]:
domains_in_interest = ["taxi", "restaurant", "attraction", "hotel", "train"]

In [3]:
def get_domains_samples(data, domains_in_interest):
    domains_samples = {domain: list() for domain in domains_in_interest}
    processed_data = dict()
    for dialogue_content in data:
        for domain in dialogue_content["domains"]:
            if domain in domains_in_interest:
                domains_samples[domain].append(dialogue_content["dialogue_idx"])
                processed_data[dialogue_content["dialogue_idx"]] = dialogue_content
    return domains_samples, processed_data

In [4]:
domains_samples, data = get_domains_samples(data, domains_in_interest)

In [ ]:
total_samples = set()
total_length = 0

for domain, samples in domains_samples.items():
    total_samples = total_samples.union(set(samples))
    total_length += len(samples)
# total_samples = list(total_samples)
total_samples = sorted(list(total_samples))
random.seed(10)
total_samples = random.sample(total_samples, 100)

print(len(total_samples))
print(total_samples[:5])
print(total_length)

999
['MUL1274.json', 'MUL1806.json', 'PMUL3599.json', 'MUL1848.json', 'PMUL2746.json']
1915


In [ ]:
client_config = {
    "client": AzureOpenAI(
        azure_endpoint = "https://uiuc-convai.openai.azure.com/",
        api_key = getpass("🔑 Enter your OpenAI API key: "),
        api_version="2024-02-15-preview"
    ),
    "model": "gpt-4o-mini",
}

In [7]:
offline_evaluator = evaluator(
    data,
    old_data,
    total_samples,
    prompt=eval_prompt_v3,
    # prompt=eval_prompt_v3_reduced_objective,
    client_config=client_config,
    online=False
)

In [8]:
save_dir = "eval_result-38.json"

offline_evaluator.evaluate()
offline_evaluator.save_result(save_dir)

100%|██████████| 999/999 [17:15:07<00:00, 62.17s/it]    
